Run this cell once

In [ ]:
%%shell
set -e
wget -nv https://raw.githubusercontent.com/odow/SESO2023/main/install_colab.sh -O /tmp/install_colab.sh
bash /tmp/install_colab.sh  # Takes ~ 2 minutes

Then refresh the page... and run this cell once

In [ ]:
import Downloads, Pkg
Downloads.download("https://raw.githubusercontent.com/odow/SESO2023/main/Project.toml", "/tmp/Project.toml")
Pkg.activate("/tmp/Project.toml")
Pkg.instantiate()  # Can take ~ 7 minutes

# Example: Two-stage Newsvendor

The purpose of this tutorial is to demonstrate how to model and solve a
two-stage stochastic program.

It is based on the [Two stage stochastic programs](https://jump.dev/JuMP.jl/dev/tutorials/applications/two_stage_stochastic/)
tutorial in JuMP.

This tutorial uses the following packages

In [ ]:
using JuMP
using SDDP
import Distributions
import HiGHS
import Plots
import StatsPlots
import Statistics

## Background

The data for this problem is:

In [ ]:
D = Distributions.TriangularDist(150.0, 250.0, 200.0)
N = 100
d = sort!(rand(D, N));
Ω = 1:N
P = fill(1 / N, N);
StatsPlots.histogram(d; bins = 20, label = "", xlabel = "Demand")

## Policy Graph

Now we can formulate and train a policy for the two-stage newsvendor problem.

First, we need to construct the graph:

In [ ]:
graph = SDDP.LinearGraph(2)

Then, we need to write a function which builds a JuMP model for each node in
the graph:

In [ ]:
function build_subproblem(subproblem::JuMP.Model, stage::Int)
    @variable(subproblem, x >= 0, SDDP.State, initial_value = 0)
    if stage == 1
        @variable(subproblem, u_make >= 0)
        @constraint(subproblem, x.out == x.in + u_make)
        @stageobjective(subproblem, -2 * u_make)
    else
        @variable(subproblem, u_sell >= 0)
        @constraint(subproblem, u_sell <= x.in)
        @constraint(subproblem, x.out == x.in - u_sell)
        SDDP.parameterize(subproblem, d, P) do ω
            set_upper_bound(u_sell, ω)
            return
        end
        @stageobjective(subproblem, 5 * u_sell - 0.1 * x.out)
    end
    return
end

Then, we can combine the graph and the subproblem builder into a policy graph:

In [ ]:
model = SDDP.PolicyGraph(
    build_subproblem,
    graph;
    sense = :Max,
    upper_bound = 5 * maximum(d), #= some large upper bound =#
    optimizer = HiGHS.Optimizer,
)

Use `SDDP.train` to construct the policy:

In [ ]:
SDDP.train(model)

To check the first-stage buy decision, we need to obtain a decision rule for
the first-stage node `1`:

In [ ]:
first_stage_rule = SDDP.DecisionRule(model, node = 1)

Then we can evaluate it, passing in a starting point for the incoming state:

In [ ]:
solution = SDDP.evaluate(first_stage_rule; incoming_state = Dict(:x => 0.0))

The optimal value of the state variable is stored here:

In [ ]:
solution.outgoing_state[:x]

We can simplify the model construction by using `SDDP.LinearPolicyGraph`:

In [ ]:
model = SDDP.LinearPolicyGraph(
    build_subproblem;
    stages = 2,
    sense = :Max,
    upper_bound = 5 * maximum(d),
    optimizer = HiGHS.Optimizer,
)

and we can use Julia's `do` syntax to avoid writing a separate function:

In [ ]:
model = SDDP.LinearPolicyGraph(;
    stages = 2,
    sense = :Max,
    upper_bound = 5 * maximum(d),
    optimizer = HiGHS.Optimizer,
) do subproblem::JuMP.Model, stage::Int
    @variable(subproblem, x >= 0, SDDP.State, initial_value = 0)
    if stage == 1
        @variable(subproblem, u_make >= 0)
        @constraint(subproblem, x.out == x.in + u_make)
        @stageobjective(subproblem, -2 * u_make)
    else
        @variable(subproblem, u_sell >= 0)
        @constraint(subproblem, u_sell <= x.in)
        @constraint(subproblem, x.out == x.in - u_sell)
        SDDP.parameterize(subproblem, d, P) do ω
            set_upper_bound(u_sell, ω)
            return
        end
        @stageobjective(subproblem, 5 * u_sell - 0.1 * x.out)
    end
    return
end

## Simulation

Querying the decision rules is tedious. It's often more useful to simulate the
policy:

In [ ]:
SDDP.train(model)

In [ ]:
simulations = SDDP.simulate(
    model,
    10,  #= number of replications =#
    [:x, :u_sell, :u_make];  #= variables to record =#
    skip_undefined_variables = true,
);

`simulations` is a vector with 10 elements

In [ ]:
length(simulations)

and each element is a vector with two elements (one for each stage)

In [ ]:
length(simulations[1])

The first stage contains:

In [ ]:
simulations[1][1]

The second stage contains:

In [ ]:
simulations[1][2]

We can compute aggregated statistics across the simulations:

In [ ]:
objectives = map(simulations) do simulation
    return sum(data[:stage_objective] for data in simulation)
end
μ, t = SDDP.confidence_interval(objectives)
println("Simulation ci : $μ ± $t")

## Risk aversion revisited

SDDP.jl contains a number of risk measures. One example is:

In [ ]:
0.5 * SDDP.Expectation() + 0.5 * SDDP.WorstCase()

You can construct a risk-averse policy by passing a risk measure to the
`risk_measure` keyword argument of `SDDP.train`.

We can explore how the optimal decision changes with risk by creating a
function:

In [ ]:
function solve_newsvendor(risk_measure::SDDP.AbstractRiskMeasure)
    model = SDDP.LinearPolicyGraph(
        stages = 2,
        sense = :Max,
        upper_bound = 5 * maximum(d),
        optimizer = HiGHS.Optimizer,
    ) do subproblem, node
        @variable(subproblem, x >= 0, SDDP.State, initial_value = 0)
        if node == 1
            @stageobjective(subproblem, -2 * x.out)
        else
            @variable(subproblem, u_sell >= 0)
            @constraint(subproblem, u_sell <= x.in)
            @constraint(subproblem, x.out == x.in - u_sell)
            SDDP.parameterize(subproblem, d, P) do ω
                set_upper_bound(u_sell, ω)
                return
            end
            @stageobjective(subproblem, 5 * u_sell - 0.1 * x.out)
        end
        return
    end
    SDDP.train(model; risk_measure = risk_measure, print_level = 0)
    first_stage_rule = SDDP.DecisionRule(model; node = 1)
    solution = SDDP.evaluate(first_stage_rule; incoming_state = Dict(:x => 0.0))
    return solution.outgoing_state[:x]
end

Now we can see how many units a decision maker would order using `CVaR`:

In [ ]:
solve_newsvendor(SDDP.CVaR(0.4))

as well as a decision-maker who cares only about the worst-case outcome:

In [ ]:
solve_newsvendor(SDDP.WorstCase())

In general, the decision-maker will be somewhere between the two extremes.
The `SDDP.Entropic` risk measure is a risk measure that has a single
parameter that lets us explore the space of policies between the two extremes.
When the parameter is small, the measure acts like `SDDP.Expectation`,
and when it is large, it acts like `SDDP.WorstCase`.

Here is what we get if we solve our problem multiple times for different
values of the risk aversion parameter $\gamma$:

In [ ]:
Γ = [10^i for i in -4:0.5:1]
buy = [solve_newsvendor(SDDP.Entropic(γ)) for γ in Γ]
Plots.plot(
    Γ,
    buy;
    xaxis = :log,
    xlabel = "Risk aversion parameter γ",
    ylabel = "Number of pies to make",
    legend = false,
)

## Things to try

There are a number of things you can try next:

 * Experiment with different buy and sales prices
 * Experiment with different distributions of demand
 * Explore how the optimal policy changes if you use a different risk measure
 * What happens if you can only buy and sell integer numbers of newspapers?
   Try this by adding `Int` to the variable definitions:
   `@variable(subproblem, buy >= 0, Int)`
 * What happens if you use a different upper bound? Try an invalid one like
   `-100`, and a very large one like `1e12`.

## Other graphs

The two-stage newsvendor problem can be extended to other graphs. For example,
here is a three-stage graph:

In [ ]:
graph = SDDP.Graph(:root)
SDDP.add_node(graph, :make_only)
SDDP.add_node(graph, :sell_only)
SDDP.add_node(graph, :make_and_sell)
SDDP.add_edge(graph, :root => :make_only, 1.0)
SDDP.add_edge(graph, :make_only => :make_and_sell, 1.0)
SDDP.add_edge(graph, :make_and_sell => :sell_only, 1.0)
graph

with the model formulation:

In [ ]:
model = SDDP.PolicyGraph(
    graph;
    sense = :Max,
    upper_bound = 2 * 5 * maximum(d),
    optimizer = HiGHS.Optimizer,
) do subproblem::JuMP.Model, node::Symbol
    @variable(subproblem, x >= 0, SDDP.State, initial_value = 0)
    if node == :make_only
        @variable(subproblem, u_make >= 0)
        @constraint(subproblem, x.out == x.in + u_make)
        @stageobjective(subproblem, -2 * u_make)
    elseif node == :sell_only
        @variable(subproblem, u_sell >= 0)
        @constraint(subproblem, u_sell <= x.in)
        @constraint(subproblem, x.out == x.in - u_sell)
        SDDP.parameterize(subproblem, d, P) do ω
            set_upper_bound(u_sell, ω)
            return
        end
        @stageobjective(subproblem, 5 * u_sell - 0.1 * x.out)
    else
        @assert node == :make_and_sell
        @variable(subproblem, u_sell >= 0)
        @variable(subproblem, u_make >= 0)
        @constraint(subproblem, u_sell <= x.in)
        @constraint(subproblem, x.out == x.in - u_sell + u_make)
        SDDP.parameterize(subproblem, d, P) do ω
            set_upper_bound(u_sell, ω)
            return
        end
        @stageobjective(subproblem, 5 * u_sell - 0.1 * x.out - 2 * u_make)
    end
    return
end

SDDP.train(model)
simulations = SDDP.simulate(
    model,
    10,  #= number of replications =#
    [:x, :u_sell, :u_make];  #= variables to record =#
    skip_undefined_variables = true,
);

In [ ]:
simulations[1][1]

In [ ]:
simulations[1][2]

In [ ]:
simulations[1][3]

Try changing the graph. What happens if you add a cycle?

# Example: deterministic to stochastic

_This tutorial was written by Oscar Dowson and Andy Philpott for the 2023
Winter School "Planning under Uncertainty in Energy Markets," held March 26 to
31 in Geilo, Norway._

[Download this tutorial as a notebook.](../../example_reservoir.ipynb)

The purpose of this tutorial is to explain how we can go from a deterministic
time-staged optimal control model in JuMP to a multistage stochastic
optimization model in `SDDP.jl`. As a motivating problem, we consider the
hydro-thermal problem with a single reservoir.

For variations on this problem, see the examples and tutorials:

 * An introduction to SDDP.jl
 * Hydro-thermal scheduling
 * Hydro valleys
 * Infinite horizon hydro-thermal

## Packages

This tutorial requires the following packages:

In [ ]:
using JuMP
using SDDP
import CSV
import DataFrames
import HiGHS
import Plots

## Data

First, we need some data for the problem. For this tutorial, we'll write CSV
files to a temporary directory from Julia. If you have an existing file, you
could change the filename to point to that instead.

In [ ]:
dir = mktempdir()
filename = joinpath(dir, "example_reservoir.csv")

Here is the data

In [ ]:
csv_data = """
week,inflow,demand,cost
1,3,7,10.2\n2,2,7.1,10.4\n3,3,7.2,10.6\n4,2,7.3,10.9\n5,3,7.4,11.2\n
6,2,7.6,11.5\n7,3,7.8,11.9\n8,2,8.1,12.3\n9,3,8.3,12.7\n10,2,8.6,13.1\n
11,3,8.9,13.6\n12,2,9.2,14\n13,3,9.5,14.5\n14,2,9.8,14.9\n15,3,10.1,15.3\n
16,2,10.4,15.8\n17,3,10.7,16.2\n18,2,10.9,16.6\n19,3,11.2,17\n20,3,11.4,17.4\n
21,3,11.6,17.7\n22,2,11.7,18\n23,3,11.8,18.3\n24,2,11.9,18.5\n25,3,12,18.7\n
26,2,12,18.9\n27,3,12,19\n28,2,11.9,19.1\n29,3,11.8,19.2\n30,2,11.7,19.2\n
31,3,11.6,19.2\n32,2,11.4,19.2\n33,3,11.2,19.1\n34,2,10.9,19\n35,3,10.7,18.9\n
36,2,10.4,18.8\n37,3,10.1,18.6\n38,2,9.8,18.5\n39,3,9.5,18.4\n40,3,9.2,18.2\n
41,2,8.9,18.1\n42,3,8.6,17.9\n43,2,8.3,17.8\n44,3,8.1,17.7\n45,2,7.8,17.6\n
46,3,7.6,17.5\n47,2,7.4,17.5\n48,3,7.3,17.5\n49,2,7.2,17.5\n50,3,7.1,17.6\n
51,3,7,17.7\n52,3,7,17.8\n
"""
write(filename, csv_data);

And here we read it into a DataFrame:

In [ ]:
data = CSV.read(filename, DataFrames.DataFrame)

It's easier to visualize the data if we plot it:

In [ ]:
Plots.plot(
    Plots.plot(data[!, :inflow], ylabel = "Inflow"),
    Plots.plot(data[!, :demand], ylabel = "Demand"),
    Plots.plot(data[!, :cost], ylabel = "Cost", xlabel = "Week");
    layout = (3, 1),
    legend = false,
)

The number of weeks will be useful later:

In [ ]:
T = size(data, 1)

## Deterministic JuMP model

To start, we construct a deterministic model in pure JuMP.

Create a JuMP model, using `HiGHS` as the optimizer:

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

`x_storage[t]`: the amount of water in the reservoir at the start of stage `t`:

In [ ]:
reservoir_max = 320.0
@variable(model, 0 <= x_storage[1:T+1] <= reservoir_max)

We need an initial condition for `x_storage[1]`. Fix it to 300 units:

In [ ]:
reservoir_initial = 300
fix(x_storage[1], reservoir_initial; force = true)

`u_flow[t]`: the amount of water to flow through the turbine in stage `t`:

In [ ]:
flow_max = 12
@variable(model, 0 <= u_flow[1:T] <= flow_max)

`u_spill[t]`: the amount of water to spill from the reservoir in stage `t`,
bypassing the turbine:

In [ ]:
@variable(model, 0 <= u_spill[1:T])

`u_thermal[t]`: the amount of thermal generation in stage `t`:

In [ ]:
@variable(model, 0 <= u_thermal[1:T])

`ω_inflow[t]`: the amount of inflow to the reservoir in stage `t`:

In [ ]:
@variable(model, ω_inflow[1:T])

For this model, our inflow is fixed, so we fix it to the data we have:

In [ ]:
for t in 1:T
    fix(ω_inflow[t], data[t, :inflow])
end

The water balance constraint says that the water in the reservoir at the start
of stage `t+1` is the water in the reservoir at the start of stage `t`, less
the amount flowed through the turbine, `u_flow[t]`, less the amount spilled,
`u_spill[t]`, plus the amount of inflow, `ω_inflow[t]`, into the reservoir:

In [ ]:
@constraint(
    model,
    [t in 1:T],
    x_storage[t+1] == x_storage[t] - u_flow[t] - u_spill[t] + ω_inflow[t],
)

We also need a `supply = demand` constraint. In practice, the units of this
would be in MWh, and there would be a conversion factor between the amount of
water flowing through the turbine and the power output. To simplify, we assume
that power and water have the same units, so that one "unit" of demand is
equal to one "unit" of the reservoir `x_storage[t]`:

In [ ]:
@constraint(model, [t in 1:T], u_flow[t] + u_thermal[t] == data[t, :demand])

Our objective is to minimize the cost of thermal generation:

In [ ]:
@objective(model, Min, sum(data[t, :cost] * u_thermal[t] for t in 1:T))

Let's optimize and check the solution

In [ ]:
optimize!(model)
solution_summary(model)

The total cost is:

In [ ]:
objective_value(model)

Here's a plot of demand and generation:

In [ ]:
Plots.plot(data[!, :demand]; label = "Demand", xlabel = "Week")
Plots.plot!(value.(u_thermal), label = "Thermal")
Plots.plot!(value.(u_flow), label = "Hydro")

And here's the storage over time:

In [ ]:
Plots.plot(value.(x_storage); label = "Storage", xlabel = "Week")

## Deterministic SDDP model

For the next step, we show how to decompose our JuMP model into SDDP.jl. It
should obtain the same solution.

In [ ]:
model = SDDP.LinearPolicyGraph(
    stages = T,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do sp, t
    @variable(
        sp,
        0 <= x_storage <= reservoir_max,
        SDDP.State,
        initial_value = reservoir_initial,
    )
    @variable(sp, 0 <= u_flow <= flow_max)
    @variable(sp, 0 <= u_thermal)
    @variable(sp, 0 <= u_spill)
    @variable(sp, ω_inflow)
    fix(ω_inflow, data[t, :inflow])
    @constraint(sp, x_storage.out == x_storage.in - u_flow - u_spill + ω_inflow)
    @constraint(sp, u_flow + u_thermal == data[t, :demand])
    @stageobjective(sp, data[t, :cost] * u_thermal)
    return
end

Can you see how the JuMP model maps to this syntax? We have created a
`SDDP.LinearPolicyGraph` with `T` stages, we're minimizing, and we're
using `HiGHS.Optimizer` as the optimizer.

A few bits might be non-obvious:

* We need to provide a lower bound for the objective function. Since our costs
  are always positive, a valid lower bound for the total cost is `0.0`.
* We define `x_storage` as a state variable using `SDDP.State`. A state
  variable is any variable that flows through time, and for which we need to
  know the value of it in stage `t-1` to compute the best action in stage `t`.
  The state variable `x_storage` is actually two decision variables,
  `x_storage.in` and `x_storage.out`, which represent `x_storage[t]` and
  `x_storage[t+1]` respectively.
* We need to use `@stageobjective` instead of `@objective`.

Instead of calling `JuMP.optimize!`, SDDP.jl uses a `train` method. With our
machine learning hat on, you can think of SDDP.jl as training a function for
each stage that accepts the current reservoir state as input and returns the
optimal actions as output. It is also an iterative algorithm, so we need to
specify when it should terminate:

In [ ]:
SDDP.train(model; iteration_limit = 10)

As a quick sanity check, did we get the same cost as our JuMP model?

In [ ]:
SDDP.calculate_bound(model)

That's good. Next, to check the value of the decision variables. This isn't as
straight forward as our JuMP model. Instead, we need to _simulate_ the policy,
and then extract the values of the decision variables from the results of the
simulation.

Since our model is deterministic, we need only 1 replication of the
simulation, and we want to record the values of the `x_storage`, `u_flow`, and
`u_thermal` variables:

In [ ]:
simulations = SDDP.simulate(
    model,
    1,  # Number of replications
    [:x_storage, :u_flow, :u_thermal],
);

The `simulations` vector is too big to show. But it contains one element for
each replication, and each replication contains one dictionary for each stage.

For example, the data corresponding to the tenth stage in the first
replication is:

In [ ]:
simulations[1][10]

Let's grab the trace of the `u_thermal` and `u_flow` variables in the first
replication, and then plot them:

In [ ]:
r_sim = [sim[:u_thermal] for sim in simulations[1]]
u_sim = [sim[:u_flow] for sim in simulations[1]]

Plots.plot(data[!, :demand]; label = "Demand", xlabel = "Week")
Plots.plot!(r_sim, label = "Thermal")
Plots.plot!(u_sim, label = "Hydro")

Perfect. That's the same as we got before.

Now let's look at `x_storage`. This is a little more complicated, because we
need to grab the outgoing value of the state variable in each stage:

In [ ]:
x_sim = [sim[:x_storage].out for sim in simulations[1]]

Plots.plot(x_sim; label = "Storage", xlabel = "Week")

## Stochastic SDDP model

Now we add some randomness to our model. In each stage, we assume that the
inflow could be: 2 units lower, with 30% probability; the same as before, with
40% probability; or 5 units higher, with 30% probability.

In [ ]:
model = SDDP.LinearPolicyGraph(
    stages = T,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do sp, t
    @variable(
        sp,
        0 <= x_storage <= reservoir_max,
        SDDP.State,
        initial_value = reservoir_initial,
    )
    @variable(sp, 0 <= u_flow <= flow_max)
    @variable(sp, 0 <= u_thermal)
    @variable(sp, 0 <= u_spill)
    @variable(sp, ω_inflow)
    # <--- This bit is new
    Ω, P = [-2, 0, 5], [0.3, 0.4, 0.3]
    SDDP.parameterize(sp, Ω, P) do ω
        fix(ω_inflow, data[t, :inflow] + ω)
        return
    end
    # --->
    @constraint(sp, x_storage.out == x_storage.in - u_flow - u_spill + ω_inflow)
    @constraint(sp, u_flow + u_thermal == data[t, :demand])
    @stageobjective(sp, data[t, :cost] * u_thermal)
    return
end

Can you see the differences?

Let's train our new model. We need more iterations because of the
stochasticity:

In [ ]:
SDDP.train(model; iteration_limit = 100)

Now simulate the policy. This time we do 100 replications because the policy
is now stochastic instead of deterministic:

In [ ]:
simulations =
    SDDP.simulate(model, 100, [:x_storage, :u_flow, :u_thermal, :ω_inflow]);

And let's plot the use of thermal generation in each replication:

In [ ]:
plot = Plots.plot(data[!, :demand]; label = "Demand", xlabel = "Week")
for simulation in simulations
    Plots.plot!(plot, [sim[:u_thermal] for sim in simulation], label = "")
end
plot

Viewing an interpreting static plots like this is difficult, particularly as
the number of simulations grows. SDDP.jl includes an interactive
`SpaghettiPlot` that makes things easier:

In [ ]:
plot = SDDP.SpaghettiPlot(simulations)
SDDP.add_spaghetti(plot; title = "Storage") do sim
    return sim[:x_storage].out
end
SDDP.add_spaghetti(plot; title = "Hydro") do sim
    return sim[:u_flow]
end
SDDP.add_spaghetti(plot; title = "Inflow") do sim
    return sim[:ω_inflow]
end
SDDP.plot(
    plot,
    "spaghetti_plot.html";
    # We need this to build the documentation. Set to true if running locally.
    open = false,
)

```@raw html
<iframe src="../spaghetti_plot.html" style="width:100%;height:500px;"></iframe>
```

**Info**
    If you have trouble viewing the plot, you can
    [open it in a new window](../../spaghetti_plot.html).

## Next steps

Take this model and modify it following the suggestions below. The SDDP.jl
documentation has a range of similar examples and hints for how to achieve
them.

### Terminal value functions

The model ends with an empty reservoir. That isn't ideal for the following
year. Can you modify the objective in the final stage to encourage ending the
year with a full reservoir?

You might write some variation of:

```julia
if t == 52
    @variable(sp, terminal_cost_function >= 0)
    @constraint(sp, terminal_cost_function >= reservoir_initial - x_storage.out)
    @stageobjective(sp, data[t, :cost] * u_thermal + terminal_cost_function)
else
    @stageobjective(sp, data[t, :cost] * u_thermal)
end
```

### Higher fidelity modeling

Our model is very basic. There are many aspects that we could improve:

* Instead of hard-coding a terminal value function, can you solve an infinite
  horizon model? What are the differences?

* Can you add a second reservoir to make a river chain?

* Can you modify the problem and data to use proper units, including a
  conversion between the volume of water flowing through the turbine and the
  electrical power output?

* Can you add random demand or cost data as well as inflows?

### Algorithmic considerations

The algorithm implemented by SDDP.jl has a number of tuneable parameters:

* Try using a different lower bound. What happens if it is too low, or too
  high?

* Was our stopping rule correct? What happens if we use fewer or more
  iterations? What other stopping rules could you try?

* Can you add a risk measure to make the policy risk-averse?

# Example: the milk producer

The purpose of this tutorial is to demonstrate how to fit a Markovian policy
graph to a univariate stochastic process.

This tutorial uses the following packages:

In [ ]:
using SDDP
import HiGHS
import Plots

## Background

A company produces milk for sale on a spot market each month. The quantity of
milk they produce is uncertain, and so too is the price on the spot market.
The company can store unsold milk in a stockpile of dried milk powder.

The spot price is determined by an auction system, and so varies from month to
month, but demonstrates serial correlation. In each auction, there is
sufficient demand that the milk producer finds a buyer for all their
milk, regardless of the quantity they supply. Furthermore, the spot price
is independent of the milk producer (they are a small player in the market).

The spot price is highly volatile, and is the result of a process that is out
of the control of the company. To counteract their price risk, the company
engages in a forward contracting programme.

The forward contracting programme is a deal for physical milk four months in
the future.

The futures price is the current spot price, plus some forward contango (the
buyers gain certainty that they will receive the milk in the future).

In general, the milk company should forward contract (since they reduce
their price risk), however they also have production risk. Therefore, it may
be the case that they forward contract a fixed amount, but find that they do
not produce enough milk to meet the fixed demand. They are then forced to
buy additional milk on the spot market.

The goal of the milk company is to choose the extent to which they forward
contract in order to maximise (risk-adjusted) revenues, whilst managing their
production risk.

## A stochastic process for price

It is outside the scope of this tutorial, but assume that we have gone away
and analysed historical data to fit a stochastic process to the sequence of
monthly auction spot prices.

One plausible model is a multiplicative auto-regressive model of order one,
where the white noise term is modeled by a finite distribution of empirical
residuals. We can simulate this stochastic process as follows:

In [ ]:
function simulator()
    residuals = [0.0987, 0.199, 0.303, 0.412, 0.530, 0.661, 0.814, 1.010, 1.290]
    residuals = 0.1 * vcat(-residuals, 0.0, residuals)
    scenario = zeros(12)
    y, μ, α = 4.5, 6.0, 0.05
    for t in 1:12
        y = exp((1 - α) * log(y) + α * log(μ) + rand(residuals))
        scenario[t] = clamp(y, 3.0, 9.0)
    end
    return scenario
end

simulator()

It may be helpful to visualize a number of simulations of the price process:

In [ ]:
plot = Plots.plot(
    [simulator() for _ in 1:500];
    color = "gray",
    opacity = 0.2,
    legend = false,
    xlabel = "Month",
    ylabel = "Price [\$/kg]",
    xlims = (1, 12),
    ylims = (3, 9),
)

The prices gradually revert to the mean of \$6/kg, and there is high
volatility.

We can't incorporate this price process directly into SDDP.jl, but we can fit
a `SDDP.MarkovianGraph` directly from the simulator:

In [ ]:
graph = SDDP.MarkovianGraph(simulator; budget = 30, scenarios = 10_000);
nothing  # hide

Here `budget` is the number of nodes in the policy graph, and `scenarios` is
the number of simulations to use when estimating the transition probabilities.

The graph contains too many nodes to be show, but we can plot it:

In [ ]:
for ((t, price), edges) in graph.nodes
    for ((t′, price′), probability) in edges
        Plots.plot!(
            plot,
            [t, t′],
            [price, price′];
            color = "red",
            width = 3 * probability,
        )
    end
end

plot

That looks okay. Try changing `budget` and `scenarios` to see how different
Markovian policy graphs can be created.

## Model

Now that we have a Markovian graph, we can build the model. See if you can
work out how we arrived at this formulation by reading the background
description. Do all the variables and constraints make sense?

In [ ]:
model = SDDP.PolicyGraph(
    graph;
    sense = :Max,
    upper_bound = 1e2,
    optimizer = HiGHS.Optimizer,
) do sp, node
    # Decompose the node into the month (::Int) and spot price (::Float64)
    t, price = node::Tuple{Int,Float64}
    # Transactions on the futures market cost 0.01
    c_transaction = 0.01
    # It costs the company +50% to buy milk on the spot market and deliver to
    # their customers
    c_buy_premium = 1.5
    # Buyer is willing to pay +5% for certainty
    c_contango = 1.05
    # Distribution of production
    Ω_production = range(0.1, 0.2; length = 5)
    c_max_production = 12 * maximum(Ω_production)
    # x_stock: quantity of milk in stock pile
    @variable(sp, 0 <= x_stock, SDDP.State, initial_value = 0)
    # x_forward[i]: quantity of milk for delivery in i months
    @variable(sp, 0 <= x_forward[1:4], SDDP.State, initial_value = 0)
    # u_spot_sell: quantity of milk to sell on spot market
    @variable(sp, 0 <= u_spot_sell <= c_max_production)
    # u_spot_buy: quantity of milk to buy on spot market
    @variable(sp, 0 <= u_spot_buy <= c_max_production)
    # u_spot_buy: quantity of milk to sell on futures market
    c_max_futures = t <= 8 ? c_max_production : 0.0
    @variable(sp, 0 <= u_forward_sell <= c_max_futures)
    # ω_production: production random variable
    @variable(sp, ω_production)
    # Forward contracting constraints:
    @constraint(sp, [i in 1:3], x_forward[i].out == x_forward[i+1].in)
    @constraint(sp, x_forward[4].out == u_forward_sell)
    # Stockpile balance constraint
    @constraint(
        sp,
        x_stock.out ==
        x_stock.in + ω_production + u_spot_buy - x_forward[1].in - u_spot_sell
    )
    # The random variables. `price` comes from the Markov node
    Ω = [(price, p) for p in Ω_production]
    SDDP.parameterize(sp, Ω) do ω::Tuple{Float64,Float64}
        # Fix the ω_production variable
        fix(ω_production, ω[2])
        @stageobjective(
            sp,
            # Sales on spot market
            ω[1] * (u_spot_sell - c_buy_premium * u_spot_buy) +
            # Sales on futures smarket
            (ω[1] * c_contango - c_transaction) * u_forward_sell
        )
        return
    end
    return
end

## Training a policy

Now we have a model, we train a policy. The `SDDP.SimulatorSamplingScheme`
is used in the forward pass. It generates an out-of-sample sequence of prices
using `simulator` and traverses the closest sequence of nodes in the policy
graph. When calling `SDDP.parameterize` for each subproblem, it uses
the new out-of-sample price instead of the price associated with the Markov
node.

In [ ]:
SDDP.train(
    model;
    time_limit = 20,
    risk_measure = 0.5 * SDDP.Expectation() + 0.5 * SDDP.AVaR(0.25),
    sampling_scheme = SDDP.SimulatorSamplingScheme(simulator),
)

**Warning**
    We're intentionally terminating the training early so that the
    documentation doesn't take too long to build. If you run this example
    locally, increase the time limit.

## Simulating the policy

When simulating the policy, we can also use the
`SDDP.SimulatorSamplingScheme`.

In [ ]:
simulations = SDDP.simulate(
    model,
    200,
    Symbol[:x_stock, :u_forward_sell, :u_spot_sell, :u_spot_buy];
    sampling_scheme = SDDP.SimulatorSamplingScheme(simulator),
);
nothing  # hide

To show how the sampling scheme uses the new out-of-sample price instead of
the price associated with the Markov node, compare the index of the Markov
state visited in stage 12 of the first simulation:

In [ ]:
simulations[1][12][:node_index]

to the realization of the noise `(price, ω)` passed to `SDDP.parameterize`:

In [ ]:
simulations[1][12][:noise_term]

## Visualizing the policy

Finally, we can plot the policy to gain insight (although note that we
terminated the training early, so we should run the re-train the policy for
more iterations before making too many judgements).

In [ ]:
plot = Plots.plot(
    SDDP.publication_plot(simulations; title = "x_stock.out") do data
        return data[:x_stock].out
    end,
    SDDP.publication_plot(simulations; title = "u_forward_sell") do data
        return data[:u_forward_sell]
    end,
    SDDP.publication_plot(simulations; title = "u_spot_buy") do data
        return data[:u_spot_buy]
    end,
    SDDP.publication_plot(simulations; title = "u_spot_sell") do data
        return data[:u_spot_sell]
    end;
    layout = (2, 2),
)

## Next steps

* Train the policy for longer. What do you observe?
* Try creating different Markovian graphs. What happens if you add more nodes?
* Try different risk measures

# Introductory theory

**Note**
    This tutorial is aimed at advanced undergraduates or early-stage graduate
    students. You don't need prior exposure to stochastic programming!
    (Indeed, it may be better if you don't, because our approach is
    non-standard in the literature.)

    This tutorial is also a living document. If parts are unclear, please
    [open an issue](https://github.com/odow/SDDP.jl/issues/new) so it can be
    improved!

This tutorial will teach you how the stochastic dual dynamic programming
algorithm works by implementing a simplified version of the algorithm.

Our implementation is very much a "vanilla" version of SDDP; it doesn't have
(m)any fancy computational tricks (e.g., the ones included in SDDP.jl) that
you need to code a performant or stable version that will work on realistic
instances. However, our simplified implementation will work on arbitrary
policy graphs, including those with cycles such as infinite horizon problems!

**Packages**

This tutorial uses the following packages. For clarity, we call
`import PackageName` so that we must prefix `PackageName.` to all functions
and structs provided by that package. Everything not prefixed is either part
of base Julia, or we wrote it.

In [ ]:
import ForwardDiff
import HiGHS
import JuMP
import Statistics

**Tip**
    You can follow along by installing the above packages, and copy-pasting
    the code we will write into a Julia REPL. Alternatively, you can download
    the Julia `.jl` file which created this tutorial [from GitHub](https://github.com/odow/SDDP.jl/blob/master/docs/src/tutorial/21_theory_intro.jl).

## Preliminaries: background theory

Start this tutorial by reading An introduction to SDDP.jl, which
introduces the necessary notation and vocabulary that we need for this
tutorial.

## Preliminaries: Kelley's cutting plane algorithm

Kelley's cutting plane algorithm is an iterative method for minimizing convex
functions. Given a convex function $f(x)$, Kelley's constructs an
under-approximation of the function at the minimum by a set of first-order
Taylor series approximations (called **cuts**) constructed at a set of points
$k = 1,\ldots,K$:
$$
\begin{aligned}
f^K = \min\limits_{\theta \in \mathbb{R}, x \in \mathbb{R}^N} \;\; & \theta\\
& \theta \ge f(x_k) + \frac{d}{dx}f(x_k)^\top (x - x_k),\quad k=1,\ldots,K\\
& \theta \ge M,
\end{aligned}
$$
where $M$ is a sufficiently large negative number that is a lower bound for
$f$ over the domain of $x$.

Kelley's cutting plane algorithm is a structured way of choosing points $x_k$
to visit, so that as more cuts are added:
$$
\lim_{K \rightarrow \infty} f^K = \min\limits_{x \in \mathbb{R}^N} f(x)
$$
However, before we introduce the algorithm, we need to introduce some bounds.

### Bounds

By convexity, $f^K \le f(x)$ for all $x$. Thus, if $x^*$ is a minimizer of
$f$, then at any point in time we can construct a lower bound for $f(x^*)$ by
solving $f^K$.

Moreover, we can use the primal solutions $x_k^*$ returned by solving $f^k$ to
evaluate $f(x_k^*)$ to generate an upper bound.

Therefore, $f^K \le f(x^*) \le \min\limits_{k=1,\ldots,K} f(x_k^*)$.

When the lower bound is sufficiently close to the upper bound, we can
terminate the algorithm and declare that we have found an solution that is
close to optimal.

### Implementation

Here is pseudo-code fo the Kelley algorithm:

1. Take as input a convex function $f(x)$ and a iteration limit $K_{max}$.
   Set $K = 0$, and initialize $f^K$. Set $lb = -\infty$ and $ub = \infty$.
2. Solve $f^K$ to obtain a candidate solution $x_{K+1}$.
3. Update $lb = f^K$ and $ub = \min\{ub, f(x_{K+1})\}$.
4. Add a cut $\theta \ge f(x_{K+1}) + \frac{d}{dx}f\left(x_{K+1}\right)^\top (x - x_{K+1})$ to form $f^{K+1}$.
5. Increment $K$.
6. If $K = K_{max}$ or $|ub - lb| < \epsilon$, STOP, otherwise, go to step 2.

And here's a complete implementation:

In [ ]:
function kelleys_cutting_plane(
    # The function to be minimized.
    f::Function,
    # The gradient of `f`. By default, we use automatic differentiation to
    # compute the gradient of f so the user doesn't have to!
    dfdx::Function = x -> ForwardDiff.gradient(f, x);
    # The number of arguments to `f`.
    input_dimension::Int,
    # A lower bound for the function `f` over its domain.
    lower_bound::Float64,
    # The number of iterations to run Kelley's algorithm for before stopping.
    iteration_limit::Int,
    # The absolute tolerance ϵ to use for convergence.
    tolerance::Float64 = 1e-6,
)
    # Step (1):
    K = 0
    model = JuMP.Model(HiGHS.Optimizer)
    JuMP.set_silent(model)
    JuMP.@variable(model, θ >= lower_bound)
    JuMP.@variable(model, x[1:input_dimension])
    JuMP.@objective(model, Min, θ)
    x_k = fill(NaN, input_dimension)
    lower_bound, upper_bound = -Inf, Inf
    while true
        # Step (2):
        JuMP.optimize!(model)
        x_k .= JuMP.value.(x)
        # Step (3):
        lower_bound = JuMP.objective_value(model)
        upper_bound = min(upper_bound, f(x_k))
        println("K = $K : $(lower_bound) <= f(x*) <= $(upper_bound)")
        # Step (4):
        JuMP.@constraint(model, θ >= f(x_k) + dfdx(x_k)' * (x .- x_k))
        # Step (5):
        K = K + 1
        # Step (6):
        if K == iteration_limit
            println("-- Termination status: iteration limit --")
            break
        elseif abs(upper_bound - lower_bound) < tolerance
            println("-- Termination status: converged --")
            break
        end
    end
    println("Found solution: x_K = ", x_k)
    return
end

Let's run our algorithm to see what happens:

In [ ]:
kelleys_cutting_plane(
    input_dimension = 2,
    lower_bound = 0.0,
    iteration_limit = 20,
) do x
    return (x[1] - 1)^2 + (x[2] + 2)^2 + 1.0
end

**Warning**
    It's hard to choose a valid lower bound! If you choose one too loose, the
    algorithm can take a long time to converge. However, if you choose one so
    tight that $M > f(x^*)$, then you can obtain a suboptimal solution. For a
    deeper discussion of the implications for SDDP.jl, see
    Choosing an initial bound.

## Preliminaries: approximating the cost-to-go term

In the background theory section, we discussed how you could formulate an
optimal policy to a multistage stochastic program using the dynamic
programming recursion:
$$
\begin{aligned}
V_i(x, \omega) = \min\limits_{\bar{x}, x^\prime, u} \;\; & C_i(\bar{x}, u, \omega) + \mathbb{E}_{j \in i^+, \varphi \in \Omega_j}[V_j(x^\prime, \varphi)]\\
& x^\prime = T_i(\bar{x}, u, \omega) \\
& u \in U_i(\bar{x}, \omega) \\
& \bar{x} = x,
\end{aligned}
$$
where our decision rule, $\pi_i(x, \omega)$, solves this optimization problem
and returns a $u^*$ corresponding to an optimal solution. Moreover, we alluded
to the fact that the cost-to-go term (the nasty recursive expectation) makes
this problem intractable to solve.

However, if, excluding the cost-to-go term (i.e., the `SP` formulation),
$V_i(x, \omega)$ can be formulated as a linear program (this also works for
convex programs, but the math is more involved), then we can make some
progress by noticing that $x$ only appears as a right-hand side term of the
fishing constraint $\bar{x} = x$. Therefore, $V_i(x, \cdot)$ is convex with
respect to $x$ for fixed $\omega$. (If you have not seen this result before,
try to prove it.)

The fishing constraint $\bar{x} = x$ has an associated dual variable. The
economic interpretation of this dual variable is that it represents the change
in the objective function if the right-hand side $x$ is increased on the scale
of one unit. In other words, and with a slight abuse of notation, it is the
value $\frac{d}{dx} V_i(x, \omega)$. (Because $V_i$ is not differentiable, it
is a [subgradient](https://en.wikipedia.org/wiki/Subderivative) instead of a
derivative.)

If we implement the constraint $\bar{x} = x$ by setting the lower- and upper
bounds of $\bar{x}$ to $x$, then the [reduced cost](https://en.wikipedia.org/wiki/Reduced_cost)
of the decision variable $\bar{x}$ is the subgradient, and we do not need to
explicitly add the fishing constraint as a row to the constraint matrix.

**Tip**
    The subproblem can have binary and integer variables, but you'll need to
    use Lagrangian duality to compute a subgradient!

Stochastic dual dynamic programming converts this problem into a tractable
form by applying Kelley's cutting plane algorithm to the $V_j$ functions in
the cost-to-go term:
$$
\begin{aligned}
V_i^K(x, \omega) = \min\limits_{\bar{x}, x^\prime, u} \;\; & C_i(\bar{x}, u, \omega) + \theta\\
& x^\prime = T_i(\bar{x}, u, \omega) \\
& u \in U_i(\bar{x}, \omega) \\
& \bar{x} = x \\
& \theta \ge \mathbb{E}_{j \in i^+, \varphi \in \Omega_j}\left[V_j^k(x^\prime_k, \varphi) + \frac{d}{dx^\prime}V_j^k(x^\prime_k, \varphi)^\top (x^\prime - x^\prime_k)\right],\quad k=1,\ldots,K \\
& \theta \ge M.
\end{aligned}
$$

All we need now is a way of generating these cutting planes in an iterative
manner. Before we get to that though, let's start writing some code.

## Implementation: modeling

Let's make a start by defining the problem structure. Like SDDP.jl, we need a
few things:

1. A description of the structure of the policy graph: how many nodes there
   are, and the arcs linking the nodes together with their corresponding
   probabilities.
2. A JuMP model for each node in the policy graph.
3. A way to identify the incoming and outgoing state variables of each node.
4. A description of the random variable, as well as a function that we can
   call that will modify the JuMP model to reflect the realization of the
   random variable.
5. A decision variable to act as the approximated cost-to-go term.

**Warning**
    In the interests of brevity, there is minimal error checking. Think about
    all the different ways you could break the code!

### Structs

The first struct we are going to use is a `State` struct that will wrap an
incoming and outgoing state variable:

In [ ]:
struct State
    in::JuMP.VariableRef
    out::JuMP.VariableRef
end

Next, we need a struct to wrap all of the uncertainty within a node:

In [ ]:
struct Uncertainty
    parameterize::Function
    Ω::Vector{Any}
    P::Vector{Float64}
end

`parameterize` is a function which takes a realization of the random variable
$\omega\in\Omega$ and updates the subproblem accordingly. The finite discrete
random variable is defined by the vectors `Ω` and `P`, so that the random
variable takes the value `Ω[i]` with probability `P[i]`. As such, `P` should
sum to 1. (We don't check this here, but we should; we do in SDDP.jl.)

Now we have two building blocks, we can declare the structure of each node:

In [ ]:
struct Node
    subproblem::JuMP.Model
    states::Dict{Symbol,State}
    uncertainty::Uncertainty
    cost_to_go::JuMP.VariableRef
end

* `subproblem` is going to be the JuMP model that we build at each node.
* `states` is a dictionary that maps a symbolic name of a state variable to a
  `State` object wrapping the incoming and outgoing state variables in
  `subproblem`.
* `uncertainty` is an `Uncertainty` object described above.
* `cost_to_go` is a JuMP variable that approximates the cost-to-go term.

Finally, we define a simplified policy graph as follows:

In [ ]:
struct PolicyGraph
    nodes::Vector{Node}
    arcs::Vector{Dict{Int,Float64}}
end

There is a vector of nodes, as well as a data structure for the arcs. `arcs`
is a vector of dictionaries, where `arcs[i][j]` gives the probability of
transitioning from node `i` to node `j`, if an arc exists.

To simplify things, we will assume that the root node transitions to node `1`
with probability 1, and there are no other incoming arcs to node 1. Notably,
we can still define cyclic graphs though!

We also define a nice `show` method so that we don't accidentally print a
large amount of information to the screen when creating a model:

In [ ]:
function Base.show(io::IO, model::PolicyGraph)
    println(io, "A policy graph with $(length(model.nodes)) nodes")
    println(io, "Arcs:")
    for (from, arcs) in enumerate(model.arcs)
        for (to, probability) in arcs
            println(io, "  $(from) => $(to) w.p. $(probability)")
        end
    end
    return
end

### Functions

Now we have some basic types, let's implement some functions so that the user
can create a model.

First, we need an example of a function that the user will provide. Like
SDDP.jl, this takes an empty `subproblem`, and a node index, in this case
`t::Int`. You could change this function to change the model, or define a new
one later in the code.

We're going to copy the example from An introduction to SDDP.jl,
with some minor adjustments for the fact we don't have many of the bells and
whistles of SDDP.jl. You can probably see how some of the SDDP.jl
functionality like `@stageobjective` and `SDDP.parameterize`
help smooth some of the usability issues like needing to construct both the
incoming and outgoing state variables, or needing to explicitly declare
`return states, uncertainty`.

In [ ]:
function subproblem_builder(subproblem::JuMP.Model, t::Int)
    # Define the state variables. Note how we fix the incoming state to the
    # initial state variable regardless of `t`! This isn't strictly necessary;
    # it only matters that we do it for the first node.
    JuMP.@variable(subproblem, volume_in == 200)
    JuMP.@variable(subproblem, 0 <= volume_out <= 200)
    states = Dict(:volume => State(volume_in, volume_out))
    # Define the control variables.
    JuMP.@variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation >= 0
        hydro_spill >= 0
        inflow
    end)
    # Define the constraints
    JuMP.@constraints(
        subproblem,
        begin
            volume_out == volume_in + inflow - hydro_generation - hydro_spill
            demand_constraint, thermal_generation + hydro_generation == 150.0
        end
    )
    # Define the objective for each stage `t`. Note that we can use `t` as an
    # index for t = 1, 2, 3.
    fuel_cost = [50.0, 100.0, 150.0]
    JuMP.@objective(subproblem, Min, fuel_cost[t] * thermal_generation)
    # Finally, we define the uncertainty object. Because this is a simplified
    # implementation of SDDP, we shall politely ask the user to only modify the
    # constraints, and not the objective function! (Not that it changes the
    # algorithm, we just have to add more information to keep track of things.)
    uncertainty = Uncertainty([0.0, 50.0, 100.0], [1 / 3, 1 / 3, 1 / 3]) do ω
        return JuMP.fix(inflow, ω)
    end
    return states, uncertainty
end

The next function we need to define is the analog of
`SDDP.PolicyGraph`. It should be pretty readable.

In [ ]:
function PolicyGraph(
    subproblem_builder::Function;
    graph::Vector{Dict{Int,Float64}},
    lower_bound::Float64,
    optimizer,
)
    nodes = Node[]
    for t in 1:length(graph)
        # Create a model.
        model = JuMP.Model(optimizer)
        JuMP.set_silent(model)
        # Use the provided function to build out each subproblem. The user's
        # function returns a dictionary mapping `Symbol`s to `State` objects,
        # and an `Uncertainty` object.
        states, uncertainty = subproblem_builder(model, t)
        # Now add the cost-to-go terms:
        JuMP.@variable(model, cost_to_go >= lower_bound)
        obj = JuMP.objective_function(model)
        JuMP.@objective(model, Min, obj + cost_to_go)
        # If there are no outgoing arcs, the cost-to-go is 0.0.
        if length(graph[t]) == 0
            JuMP.fix(cost_to_go, 0.0; force = true)
        end
        push!(nodes, Node(model, states, uncertainty, cost_to_go))
    end
    return PolicyGraph(nodes, graph)
end

Then, we can create a model using the `subproblem_builder` function we defined
earlier:

In [ ]:
model = PolicyGraph(
    subproblem_builder;
    graph = [Dict(2 => 1.0), Dict(3 => 1.0), Dict{Int,Float64}()],
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
)

## Implementation: helpful samplers

Before we get properly coding the solution algorithm, it's also going to be
useful to have a function that samples a realization of the random variable
defined by `Ω` and `P`.

In [ ]:
function sample_uncertainty(uncertainty::Uncertainty)
    r = rand()
    for (p, ω) in zip(uncertainty.P, uncertainty.Ω)
        r -= p
        if r < 0.0
            return ω
        end
    end
    return error("We should never get here because P should sum to 1.0.")
end

**Note**
    `rand()` samples a uniform random variable in `[0, 1)`.

For example:

In [ ]:
for i in 1:3
    println("ω = ", sample_uncertainty(model.nodes[1].uncertainty))
end

It's also going to be useful to define a function that generates a random walk
through the nodes of the graph:

In [ ]:
function sample_next_node(model::PolicyGraph, current::Int)
    if length(model.arcs[current]) == 0
        # No outgoing arcs!
        return nothing
    else
        r = rand()
        for (to, probability) in model.arcs[current]
            r -= probability
            if r < 0.0
                return to
            end
        end
        # We looped through the outgoing arcs and still have probability left
        # over! This means we've hit an implicit "zero" node.
        return nothing
    end
end

For example:

In [ ]:
for i in 1:3
    # We use `repr` to print the next node, because `sample_next_node` can
    # return `nothing`.
    println("Next node from $(i) = ", repr(sample_next_node(model, i)))
end

This is a little boring, because our graph is simple. However, more
complicated graphs will generate more interesting trajectories!

## Implementation: the forward pass

Recall that, after approximating the cost-to-go term, we need a way of
generating the cuts. As the first step, we need a way of generating candidate
solutions $x_k^\prime$. However, unlike the Kelley's example, our functions
$V_j^k(x^\prime, \varphi)$ need two inputs: an outgoing state variable and a
realization of the random variable.

One way of getting these inputs is just to pick a random (feasible) value.
However, in doing so, we might pick outgoing state variables that we will
never see in practice, or we might infrequently pick outgoing state variables
that we will often see in practice. Therefore, a better way of generating the
inputs is to use a simulation of the policy, which we call the **forward**
**pass**.

The forward pass walks the policy graph from start to end, transitioning
randomly along the arcs. At each node, it observes a realization of the random
variable and solves the approximated subproblem to generate a candidate
outgoing state variable $x_k^\prime$. The outgoing state variable is passed as
the incoming state variable to the next node in the trajectory.

In [ ]:
function forward_pass(model::PolicyGraph, io::IO = stdout)
    println(io, "| Forward Pass")
    # First, get the value of the state at the root node (e.g., x_R).
    incoming_state =
        Dict(k => JuMP.fix_value(v.in) for (k, v) in model.nodes[1].states)
    # `simulation_cost` is an accumlator that is going to sum the stage-costs
    # incurred over the forward pass.
    simulation_cost = 0.0
    # We also need to record the nodes visited and resultant outgoing state
    # variables so we can pass them to the backward pass.
    trajectory = Tuple{Int,Dict{Symbol,Float64}}[]
    # Now's the meat of the forward pass: beginning at the first node:
    t = 1
    while t !== nothing
        node = model.nodes[t]
        println(io, "| | Visiting node $(t)")
        # Sample the uncertainty:
        ω = sample_uncertainty(node.uncertainty)
        println(io, "| | | ω = ", ω)
        # Parameterizing the subproblem using the user-provided function:
        node.uncertainty.parameterize(ω)
        println(io, "| | | x = ", incoming_state)
        # Update the incoming state variable:
        for (k, v) in incoming_state
            JuMP.fix(node.states[k].in, v; force = true)
        end
        # Now solve the subproblem and check we found an optimal solution:
        JuMP.optimize!(node.subproblem)
        if JuMP.termination_status(node.subproblem) != JuMP.MOI.OPTIMAL
            error("Something went terribly wrong!")
        end
        # Compute the outgoing state variables:
        outgoing_state = Dict(k => JuMP.value(v.out) for (k, v) in node.states)
        println(io, "| | | x′ = ", outgoing_state)
        # We also need to compute the stage cost to add to our
        # `simulation_cost` accumulator:
        stage_cost =
            JuMP.objective_value(node.subproblem) - JuMP.value(node.cost_to_go)
        simulation_cost += stage_cost
        println(io, "| | | C(x, u, ω) = ", stage_cost)
        # As a penultimate step, set the outgoing state of stage t and the
        # incoming state of stage t + 1, and add the node to the trajectory.
        incoming_state = outgoing_state
        push!(trajectory, (t, outgoing_state))
        # Finally, sample a new node to step to. If `t === nothing`, the
        # `while` loop will break.
        t = sample_next_node(model, t)
    end
    return trajectory, simulation_cost
end

Let's take a look at one forward pass:

In [ ]:
trajectory, simulation_cost = forward_pass(model);

## Implementation: the backward pass

From the forward pass, we obtained a vector of nodes visited and their
corresponding outgoing state variables. Now we need to refine the
approximation for each node at the candidate solution for the outgoing state
variable. That is, we need to add a new cut:
$$
\theta \ge \mathbb{E}_{j \in i^+, \varphi \in \Omega_j}\left[V_j^k(x^\prime_k, \varphi) + \frac{d}{dx^\prime}V_j^k(x^\prime_k, \varphi)^\top (x^\prime - x^\prime_k)\right]
$$
or alternatively:
$$
\theta \ge \sum\limits_{j \in i^+} \sum\limits_{\varphi \in \Omega_j} p_{ij} p_{\varphi}\left[V_j^k(x^\prime_k, \varphi) + \frac{d}{dx^\prime}V_j^k(x^\prime_k, \varphi)^\top (x^\prime - x^\prime_k)\right]
$$

It doesn't matter what order we visit the nodes to generate these cuts for.
For example, we could compute them all in parallel, using the current
approximations of $V^K_i$.

However, we can be smarter than that.

If we traverse the list of nodes visited in the forward pass in reverse, then
we come to refine the $i^{th}$ node in the trajectory, we will already have
improved the approximation of the $(i+1)^{th}$ node in the trajectory as well!
Therefore, our refinement of the $i^{th}$ node will be better than if we
improved node $i$ first, and then refined node $(i+1)$.

Because we walk the nodes in reverse, we call this the **backward pass**.

**Info**
    If you're into deep learning, you could view this as the equivalent of
    back-propagation: the forward pass pushes primal information through the
    graph (outgoing state variables), and the backward pass pulls dual
    information (cuts) back through the graph to improve our decisions on the
    next forward pass.

In [ ]:
function backward_pass(
    model::PolicyGraph,
    trajectory::Vector{Tuple{Int,Dict{Symbol,Float64}}},
    io::IO = stdout,
)
    println(io, "| Backward pass")
    # For the backward pass, we walk back up the nodes.
    for i in reverse(1:length(trajectory))
        index, outgoing_states = trajectory[i]
        node = model.nodes[index]
        println(io, "| | Visiting node $(index)")
        if length(model.arcs[index]) == 0
            # If there are no children, the cost-to-go is 0.
            println(io, "| | | Skipping node because the cost-to-go is 0")
            continue
        end
        # Create an empty affine expression that we will use to build up the
        # right-hand side of the cut expression.
        cut_expression = JuMP.AffExpr(0.0)
        # For each node j ∈ i⁺
        for (j, P_ij) in model.arcs[index]
            next_node = model.nodes[j]
            # Set the incoming state variables of node j to the outgoing state
            # variables of node i
            for (k, v) in outgoing_states
                JuMP.fix(next_node.states[k].in, v; force = true)
            end
            # Then for each realization of φ ∈ Ωⱼ
            for (pφ, φ) in zip(next_node.uncertainty.P, next_node.uncertainty.Ω)
                # Setup and solve for the realization of φ
                println(io, "| | | Solving φ = ", φ)
                next_node.uncertainty.parameterize(φ)
                JuMP.optimize!(next_node.subproblem)
                # Then prepare the cut `P_ij * pφ * [V + dVdxᵀ(x - x_k)]``
                V = JuMP.objective_value(next_node.subproblem)
                println(io, "| | | | V = ", V)
                dVdx = Dict(
                    k => JuMP.reduced_cost(v.in) for (k, v) in next_node.states
                )
                println(io, "| | | | dVdx′ = ", dVdx)
                cut_expression += JuMP.@expression(
                    node.subproblem,
                    P_ij *
                    pφ *
                    (
                        V + sum(
                            dVdx[k] * (x.out - outgoing_states[k]) for
                            (k, x) in node.states
                        )
                    ),
                )
            end
        end
        # And then refine the cost-to-go variable by adding the cut:
        c = JuMP.@constraint(node.subproblem, node.cost_to_go >= cut_expression)
        println(io, "| | | Adding cut : ", c)
    end
    return nothing
end

## Implementation: bounds

### Lower bounds

Recall from Kelley's that we can obtain a lower bound for $f(x^*)$ be
evaluating $f^K$. The analogous lower bound for a multistage stochastic
program is:

$$
\mathbb{E}_{i \in R^+, \omega \in \Omega_i}[V_i^K(x_R, \omega)] \le \min_{\pi} \mathbb{E}_{i \in R^+, \omega \in \Omega_i}[V_i^\pi(x_R, \omega)]
$$

Here's how we compute the lower bound:

In [ ]:
function lower_bound(model::PolicyGraph)
    node = model.nodes[1]
    bound = 0.0
    for (p, ω) in zip(node.uncertainty.P, node.uncertainty.Ω)
        node.uncertainty.parameterize(ω)
        JuMP.optimize!(node.subproblem)
        bound += p * JuMP.objective_value(node.subproblem)
    end
    return bound
end

**Note**
    The implementation is simplified because we assumed that there is only one
    arc from the root node, and that it pointed to the first node in the
    vector.

Because we haven't trained a policy yet, the lower bound is going to be very
bad:

In [ ]:
lower_bound(model)

### Upper bounds

With Kelley's algorithm, we could easily construct an upper bound by
evaluating $f(x_K)$. However, it is almost always intractable to evaluate an
upper bound for multistage stochastic programs due to the large number of
nodes and the nested expectations. Instead, we can perform a Monte Carlo
simulation of the policy to build a statistical estimate for the value of
$\mathbb{E}_{i \in R^+, \omega \in \Omega_i}[V_i^\pi(x_R, \omega)]$, where
$\pi$ is the policy defined by the current approximations $V^K_i$.

In [ ]:
function upper_bound(model::PolicyGraph; replications::Int)
    # Pipe the output to `devnull` so we don't print too much!
    simulations = [forward_pass(model, devnull) for i in 1:replications]
    z = [s[2] for s in simulations]
    μ = Statistics.mean(z)
    tσ = 1.96 * Statistics.std(z) / sqrt(replications)
    return μ, tσ
end

**Note**
    The width of the confidence interval is incorrect if there are cycles in
    the graph, because the distribution of simulation costs `z` is not
    symmetric. The mean is correct, however.

### Termination criteria

In Kelley's algorithm, the upper bound was deterministic. Therefore, we could
terminate the algorithm when the lower bound was sufficiently close to the
upper bound. However, our upper bound for SDDP is not deterministic; it is a
confidence interval!

Some people suggest terminating SDDP when the lower bound is contained within
the confidence interval. However, this is a poor choice because it is too easy
to generate a false positive. For example, if we use a small number of
replications then the width of the confidence will be large, and we are more
likely to terminate!

In a future tutorial (not yet written...) we will discuss termination criteria
in more depth. For now, pick a large number of iterations and train for as
long as possible.

**Tip**
    For a rule of thumb, pick a large number of iterations to train the
    policy for (e.g.,
    $10 \times |\mathcal{N}| \times \max\limits_{i\in\mathcal{N}} |\Omega_i|$)

## Implementation: the training loop

The `train` loop of SDDP just applies the forward and backward passes
iteratively, followed by a final simulation to compute the upper bound
confidence interval:

In [ ]:
function train(
    model::PolicyGraph;
    iteration_limit::Int,
    replications::Int,
    io::IO = stdout,
)
    for i in 1:iteration_limit
        println(io, "Starting iteration $(i)")
        outgoing_states, _ = forward_pass(model, io)
        backward_pass(model, outgoing_states, io)
        println(io, "| Finished iteration")
        println(io, "| | lower_bound = ", lower_bound(model))
    end
    println(io, "Termination status: iteration limit")
    μ, tσ = upper_bound(model; replications = replications)
    println(io, "Upper bound = $(μ) ± $(tσ)")
    return
end

Using our `model` we defined earlier, we can go:

In [ ]:
train(model; iteration_limit = 3, replications = 100)

Success! We trained a policy for a finite horizon multistage stochastic
program using stochastic dual dynamic programming.

## Implementation: evaluating the policy

A final step is the ability to evaluate the policy at a given point.

In [ ]:
function evaluate_policy(
    model::PolicyGraph;
    node::Int,
    incoming_state::Dict{Symbol,Float64},
    random_variable,
)
    the_node = model.nodes[node]
    the_node.uncertainty.parameterize(random_variable)
    for (k, v) in incoming_state
        JuMP.fix(the_node.states[k].in, v; force = true)
    end
    JuMP.optimize!(the_node.subproblem)
    return Dict(
        k => JuMP.value.(v) for
        (k, v) in JuMP.object_dictionary(the_node.subproblem)
    )
end

evaluate_policy(
    model;
    node = 1,
    incoming_state = Dict(:volume => 150.0),
    random_variable = 75,
)

**Note**
    The random variable can be **out-of-sample**, i.e., it doesn't have to be
    in the vector $\Omega$ we created when defining the model! This is a
    notable difference to other multistage stochastic solution methods like
    progressive hedging or using the deterministic equivalent.

## Example: infinite horizon

As promised earlier, our implementation is actually pretty general. It can
solve any multistage stochastic (linear) program defined by a policy graph,
including infinite horizon problems!

Here's an example, where we have extended our earlier problem with an arc from
node 3 to node 2 with probability 0.5. You can interpret the 0.5 as a discount
factor.

In [ ]:
model = PolicyGraph(
    subproblem_builder;
    graph = [Dict(2 => 1.0), Dict(3 => 1.0), Dict(2 => 0.5)],
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
)

Then, train a policy:

In [ ]:
train(model; iteration_limit = 3, replications = 100)

Success! We trained a policy for an infinite horizon multistage stochastic
program using stochastic dual dynamic programming. Note how some of the
forward passes are different lengths!

In [ ]:
evaluate_policy(
    model;
    node = 3,
    incoming_state = Dict(:volume => 100.0),
    random_variable = 10.0,
)

# The knapsack problem example

The purpose of this tutorial is to demonstrate how to formulate and solve a
simple optimization problem.

## Required packages

This tutorial requires the following packages:

In [ ]:
using JuMP
import HiGHS

## Formulation

The [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem)
is a classical optimization problem: given a set of items and a container with
a fixed capacity, choose a subset of items having the greatest combined
value that will fit within the container without exceeding the capacity.

The name of the problem suggests its analogy to packing for a trip,
where the baggage weight limit is the capacity and the goal is to pack the
most profitable combination of belongings.

We can formulate the knapsack problem as the integer linear program:
$$
\begin{aligned}
\max \; & \sum_{i=1}^n c_i x_i      \\
s.t. \; & \sum_{i=1}^n w_i x_i \le C, \\
        & x_i \in \{0,1\},\quad \forall i=1,\ldots,n,
\end{aligned}
$$
where $C$ is the capacity, and there is a choice between $n$ items, with
item $i$ having weight $w_i$, profit $c_i$. Decision variable $x_i$ is
equal to 1 if the item is chosen and 0 if not.

This formulation can be written more compactly as:
$$
\begin{aligned}
\max \; & c^\top x       \\
s.t. \; & w^\top x \le C \\
        & x \text{ binary }.
\end{aligned}
$$

## Data

The data for the problem consists of two vectors (one for the profits and one
for the weights) along with a capacity.

There are five objects:

In [ ]:
n = 5;

For our example, we use a capacity of 10 units:

In [ ]:
capacity = 10.0;

and the profit and cost data:

In [ ]:
profit = [5.0, 3.0, 2.0, 7.0, 4.0];
weight = [2.0, 8.0, 4.0, 2.0, 5.0];

## JuMP formulation

Let's begin constructing the JuMP model for our knapsack problem.

First, we'll create a `Model` object for holding model elements as we
construct each part. We'll also set the solver that will ultimately be called
to solve the model, once it's constructed.

In [ ]:
model = Model(HiGHS.Optimizer)

Next we need the decision variables representing which items are chosen:

In [ ]:
@variable(model, x[1:n], Bin)

We now want to constrain those variables so that their combined
weight is less than or equal to the given capacity:

In [ ]:
@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)

Finally, our objective is to maximize the combined profit of the chosen items:

In [ ]:
@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))

Let's print a human-readable description of the model and check that the model
looks as expected:

In [ ]:
print(model)

We can now solve the optimization problem and inspect the results.

In [ ]:
optimize!(model)
solution_summary(model)

The items chosen are

In [ ]:
items_chosen = [i for i in 1:n if value(x[i]) > 0.5]

## Writing a function

After working interactively, it is good practice to implement your model in a
function.

The function can be used to ensure that the model is given well-defined input
data with validation checks, and that the solution process went as expected.

In [ ]:
function solve_knapsack_problem(;
    profit::Vector{Float64},
    weight::Vector{Float64},
    capacity::Float64,
)
    n = length(weight)
    # The profit and weight vectors must be of equal length.
    @assert length(profit) == n
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n], Bin)
    @objective(model, Max, profit' * x)
    @constraint(model, weight' * x <= capacity)
    optimize!(model)
    @assert termination_status(model) == OPTIMAL
    @assert primal_status(model) == FEASIBLE_POINT
    println("Objective is: ", objective_value(model))
    println("Solution is:")
    for i in 1:n
        print("x[$i] = ", round(Int, value(x[i])))
        println(", c[$i] / w[$i] = ", profit[i] / weight[i])
    end
    chosen_items = [i for i in 1:n if value(x[i]) > 0.5]
    return return chosen_items
end

solve_knapsack_problem(; profit = profit, weight = weight, capacity = capacity)

We observe that the chosen items (1, 4, and 5) have the best
profit to weight ratio in this particular example.

## Next steps

Here are some things to try next:

* Call the function with different data. What happens as the capacity
  increases?
* What happens if the profit and weight vectors are different lengths?
* Instead of creating a binary variable with `Bin`, we could have written
  `@variable(model, 0 <= x[1:n] <= 1, Int)`. Verify that this formulation
  finds the same solution. What happens if we are allowed to take more than
  one of each item?

# The diet problem

The purpose of this tutorial is to demonstrate how to incorporate DataFrames
into a JuMP model. As an example, we use classic [Stigler diet problem](https://en.wikipedia.org/wiki/Stigler_diet).

## Required packages

This tutorial requires the following packages:

In [ ]:
using JuMP
import CSV
import DataFrames
import HiGHS
import Test

## Formulation

We wish to cook a nutritionally balanced meal by choosing the quantity of each
food $f$ to eat from a set of foods $F$ in our kitchen.

Each food $f$ has a cost, $c_f$, as well as a macro-nutrient profile
$a_{m,f}$ for each macro-nutrient $m \in M$.

Because we care about a nutritionally balanced meal, we set some minimum and
maximum limits for each nutrient, which we denote $l_m$ and $u_m$
respectively.

Furthermore, because we are optimizers, we seek the minimum cost solution.

With a little effort, we can formulate our dinner problem as the following
linear program:
$$
\begin{aligned}
\min & \sum\limits_{f \in F} c_f x_f \\
\text{s.t.}\ \ & l_m \le \sum\limits_{f \in F} a_{m,f} x_f \le u_m, && \forall m \in M \\
& x_f \ge 0, && \forall f \in F.
\end{aligned}
$$

In the rest of this tutorial, we will create and solve this problem in JuMP,
and learn what we should cook for dinner.

## Data

First, we need some data for the problem. For this tutorial, we'll write CSV
files to a temporary directory from Julia. If you have existing files, you
could change the filenames to point to them instead.

In [ ]:
dir = mktempdir()

The first file is a list of foods with their macro-nutrient profile:

In [ ]:
food_csv_filename = joinpath(dir, "diet_foods.csv")
open(food_csv_filename, "w") do io
    write(
        io,
        """
        name,cost,calories,protein,fat,sodium
        hamburger,2.49,410,24,26,730
        chicken,2.89,420,32,10,1190
        hot dog,1.50,560,20,32,1800
        fries,1.89,380,4,19,270
        macaroni,2.09,320,12,10,930
        pizza,1.99,320,15,12,820
        salad,2.49,320,31,12,1230
        milk,0.89,100,8,2.5,125
        ice cream,1.59,330,8,10,180
        """,
    )
    return
end
foods = CSV.read(food_csv_filename, DataFrames.DataFrame)

Here, $F$ is `foods.name` and $c_f$ is `foods.cost`. (We're also playing
a bit loose the term "macro-nutrient" by including calories and sodium.)

We also need our minimum and maximum limits:

In [ ]:
nutrient_csv_filename = joinpath(dir, "diet_nutrient.csv")
open(nutrient_csv_filename, "w") do io
    write(
        io,
        """
        nutrient,min,max
        calories,1800,2200
        protein,91,
        fat,0,65
        sodium,0,1779
        """,
    )
    return
end
limits = CSV.read(nutrient_csv_filename, DataFrames.DataFrame)

Protein is missing data for the maximum. Let's fix that using `coalesce`:

In [ ]:
limits.max = coalesce.(limits.max, Inf)
limits

## JuMP formulation

Now we're ready to convert our mathematical formulation into a JuMP model.

First, create a new JuMP model. Since we have a linear program, we'll use
HiGHS as our optimizer:

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

Next, we create a set of decision variables `x`, with one element for each row
in the DataFrame, and each `x` has a lower bound of `0`:

In [ ]:
@variable(model, x[foods.name] >= 0)

To simplify things later on, we store the vector as a new column `x` in the
DataFrame `foods`. Since `x` is a `DenseAxisArray`, we first need to convert
it to an `Array`:

In [ ]:
foods.x = Array(x)

Our objective is to minimize the total cost of purchasing food:

In [ ]:
@objective(model, Min, sum(foods.cost .* foods.x));

For the next component, we need to add a constraint that our total intake of
each component is within the limits contained in the `limits` DataFrame:

In [ ]:
@constraint(
    model,
    [row in eachrow(limits)],
    row.min <= sum(foods[!, row.nutrient] .* foods.x) <= row.max,
);

What does our model look like?

In [ ]:
print(model)

## Solution

Let's optimize and take a look at the solution:

In [ ]:
optimize!(model)
Test.@test primal_status(model) == FEASIBLE_POINT
Test.@test objective_value(model) ≈ 11.8288 atol = 1e-4
solution_summary(model)

We found an optimal solution. Let's see what the optimal solution is:

In [ ]:
for row in eachrow(foods)
    println(row.name, " = ", value(row.x))
end

That's a lot of milk and ice cream, and sadly, we only get `0.6` of a
hamburger.

We can also use the function `Containers.rowtable` to easily convert
the result into a DataFrame:

In [ ]:
table = Containers.rowtable(value, x; header = [:food, :quantity])
solution = DataFrames.DataFrame(table)

This makes it easy to perform analyses our solution:

In [ ]:
filter!(row -> row.quantity > 0.0, solution)

## Problem modification

JuMP makes it easy to take an existing model and modify it by adding extra
constraints. Let's see what happens if we add a constraint that we can buy at
most 6 units of milk or ice cream combined.

In [ ]:
dairy_foods = ["milk", "ice cream"]
is_dairy = map(name -> name in dairy_foods, foods.name)
dairy_constraint = @constraint(model, sum(foods[is_dairy, :x]) <= 6)
optimize!(model)
Test.@test termination_status(model) == INFEASIBLE
Test.@test primal_status(model) == NO_SOLUTION
solution_summary(model)

There exists no feasible solution to our problem. Looks like we're stuck
eating ice cream for dinner.

## Next steps

* You can delete a constraint using `delete(model, dairy_constraint)`. Can you
  add a different constraint to provide a diet with less dairy?
* Some food items (like hamburgers) are discrete. You can use `set_integer`
  to force a variable to take integer values. What happens to the solution if
  you do?

# Two-stage stochastic programs

The purpose of this tutorial is to demonstrate how to model and solve a
two-stage stochastic program.

This tutorial uses the following packages

In [ ]:
using JuMP
import Distributions
import HiGHS
import Plots
import StatsPlots
import Statistics

## Background

During the week, you are a busy practitioner of Operations Research. To escape
the drudgery of mathematics, you decide to open a side business selling creamy
mushroom pies with puff pastry. After a few weeks, it quickly becomes apparent
that operating a food business is not so easy.

The pies must be prepared in the morning, _before_ you open for the day and
can gauge the level of demand. If you bake too many, the unsold pies at the
end of the day must be discarded and you have wasted time and money on their
production. But if you bake too few, then there may be un-served customers and
you could have made more money by baking more pies.

After a few weeks of poor decision making, you decide to put your knowledge of
Operations Research to good use, starting with some data collection.

Each pie costs you \$2 to make, and you sell them at \$5 each. Disposal of an
unsold pie costs \$0.10. Based on three weeks of data collected, in which you
made 200 pies each week, you sold 150, 190, and 200 pies. Thus, as a guess,
you assume a triangular distribution of demand with a minimum of 150, a median
of 200, and a maximum of 250.

We can model this problem by a two-stage stochastic program. In the first
stage, we decide a quantity of pies to make $x$. We make this decision
before we observe the demand $d_\omega$. In the second stage, we sell
$y_\omega$ pies, and incur any costs for unsold pies.

We can formulate this problem as follows:
$$
\begin{aligned}
\max\limits_{x,y_\omega} \;\; & -2x + \mathbb{E}_\omega[5y_\omega - 0.1(x - y_\omega)] \\
  & y_\omega \le x              & \quad \forall \omega \in \Omega \\
  & 0 \le y_\omega \le d_\omega & \quad \forall \omega \in \Omega \\
  & x \ge 0.
\end{aligned}
$$

## Sample Average approximation

If the distribution of demand is continuous, then our problem has an infinite
number of variables and constraints. To form a computationally tractable
problem, we instead use a finite set of samples drawn from the distribution.
This is called sample average approximation (SAA).

In [ ]:
D = Distributions.TriangularDist(150.0, 250.0, 200.0)
N = 100
d = sort!(rand(D, N));
Ω = 1:N
P = fill(1 / N, N);
StatsPlots.histogram(d; bins = 20, label = "", xlabel = "Demand")

## JuMP model

The implementation of our two-stage stochastic program in JuMP is:

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x >= 0)
@variable(model, 0 <= y[ω in Ω] <= d[ω])
@constraint(model, [ω in Ω], y[ω] <= x)
@expression(model, z[ω in Ω], 5y[ω] - 0.1 * (x - y[ω]))
@objective(model, Max, -2x + sum(P[ω] * z[ω] for ω in Ω))
optimize!(model)
solution_summary(model)

The optimal number of pies to make is:

In [ ]:
value(x)

The distribution of total profit is:

In [ ]:
total_profit = [-2 * value(x) + value(z[ω]) for ω in Ω]

Let's plot it:

In [ ]:
"""
    bin_distribution(x::Vector{Float64}, N::Int)

A helper function that discretizes `x` into bins of width `N`.
"""
bin_distribution(x, N) = N * (floor(minimum(x) / N):ceil(maximum(x) / N))

plot = StatsPlots.histogram(
    total_profit;
    bins = bin_distribution(total_profit, 25),
    label = "",
    xlabel = "Profit [\$]",
    ylabel = "Number of outcomes",
)
μ = Statistics.mean(total_profit)
Plots.vline!(
    plot,
    [μ];
    label = "Expected profit (\$$(round(Int, μ)))",
    linewidth = 3,
)
plot

## Risk measures

A risk measure is a function which maps a random variable to a real number.
Common risk measures include the mean (expectation), median, mode, and
maximum. We need a risk measure to convert the distribution of second stage
costs into a single number that can be optimized.

Our model currently uses the expectation risk measure, but others are possible
too. One popular risk measure is the conditional value at risk (CVaR).

CVaR has a parameter $\gamma$, and it computes the expectation of the worst
$\gamma$ fraction of outcomes.

If we are maximizing, so that small outcomes are bad, the definition of CVaR
is:
$$
CVaR_{\gamma}[Z] = \max\limits_{\xi} \;\; \xi - \frac{1}{\gamma}\mathbb{E}_\omega\left[(\xi - Z)_+\right]
$$
which can be formulated as the linear program:
$$
\begin{aligned}
CVaR_{\gamma}[Z] = \max\limits_{\xi, z_\omega} \;\; & \xi - \frac{1}{\gamma}\sum P_\omega z_\omega\\
 & z_\omega \ge \xi - Z_\omega & \quad \forall \omega \\
 & z_\omega \ge 0 & \quad \forall \omega.
\end{aligned}
$$

In [ ]:
function CVaR(Z::Vector{Float64}, P::Vector{Float64}; γ::Float64)
    @assert 0 < γ <= 1
    N = length(Z)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, ξ)
    @variable(model, z[1:N] >= 0)
    @constraint(model, [i in 1:N], z[i] >= ξ - Z[i])
    @objective(model, Max, ξ - 1 / γ * sum(P[i] * z[i] for i in 1:N))
    optimize!(model)
    return objective_value(model)
end

When `γ` is `1.0`, we compute the mean of the profit:

In [ ]:
cvar_10 = CVaR(total_profit, P; γ = 1.0)

In [ ]:
Statistics.mean(total_profit)

As `γ` approaches `0.0`, we compute the worst-case (minimum) profit:

In [ ]:
cvar_00 = CVaR(total_profit, P; γ = 0.0001)

In [ ]:
minimum(total_profit)

By varying `γ` between `0` and `1` we can compute some trade-off of these two
extremes:

In [ ]:
cvar_05 = CVaR(total_profit, P; γ = 0.5)

Let's plot these outcomes on our distribution:

In [ ]:
plot = StatsPlots.histogram(
    total_profit;
    bins = bin_distribution(total_profit, 25),
    label = "",
    xlabel = "Profit [\$]",
    ylabel = "Number of outcomes",
)
Plots.vline!(
    plot,
    [cvar_10 cvar_05 cvar_00];
    label = ["γ = 1.0" "γ = 0.5" "γ = 0.0"],
    linewidth = 3,
)
plot

## Risk averse sample average approximation

Because CVaR can be formulated as a linear program, we can form a risk averse
sample average approximation model by combining the two formulations:

In [ ]:
γ = 0.4
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x >= 0)
@variable(model, 0 <= y[ω in Ω] <= d[ω])
@constraint(model, [ω in Ω], y[ω] <= x)
@expression(model, Z[ω in Ω], 5 * y[ω] - 0.1(x - y[ω]))
@variable(model, ξ)
@variable(model, z[ω in Ω] >= 0)
@constraint(model, [ω in Ω], z[ω] >= ξ - Z[ω])
@objective(model, Max, -2x + ξ - 1 / γ * sum(P[ω] * z[ω] for ω in Ω))
optimize!(model)

When $\gamma = 0.4$, the optimal number of pies to bake is:

In [ ]:
value(x)

The distribution of total profit is:

In [ ]:
risk_averse_total_profit = [value(-2x + Z[ω]) for ω in Ω]
bins = bin_distribution([total_profit; risk_averse_total_profit], 25)
plot = StatsPlots.histogram(total_profit; label = "Expectation", bins = bins)
StatsPlots.histogram!(
    plot,
    risk_averse_total_profit;
    label = "CV@R",
    bins = bins,
    alpha = 0.5,
)
plot

## Next steps

 * Try solving this problem for different numbers of samples and different
   distributions.
 * Refactor the example to avoid hard-coding the costs. What happens to the
   solution if the cost of disposing unsold pies increases?
 * Plot the optimal number of pies to make for different values of the risk
   aversion parameter $\gamma$. What is the relationship?